In [13]:
import os
import re 
import glob
import time
import numpy as np
import pandas as pd
import pandas_profiling
from collections import Counter

%autosave 60
%matplotlib inline

Autosaving every 60 seconds


In [4]:
# This is the path where you want to search
path = r'NER_PRECEDENTE_EXPORT'

# this is the extension you want to detect
extension = '.csv'
file_name_path1 = []
for root, dirs_list, files_list in os.walk(path):
    for file_name in files_list:
        if os.path.splitext(file_name)[-1] == extension:
            file_name_path = os.path.join(root, file_name)
#             print(file_name)
#             print(file_name_path)   # This is the full path of the filter file
            file_name_path1.append(file_name_path)

In [5]:
len(file_name_path1)

29412

In [7]:
%%time
# Une todos os arquivos.

frames = []
for all_files in file_name_path1:
    df = pd.read_csv(all_files,delimiter=';', na_values='NaN') # Lê o arquivo
    frames.append(df) # Adiciona esse dataframe no 'dataframe maior'
    
df_prec = pd.concat(frames).reset_index(drop=True)

CPU times: user 35.6 s, sys: 588 ms, total: 36.2 s
Wall time: 45.2 s


In [8]:
df_prec.to_csv("df_prec.csv",index=False,encoding='utf-8') # Cria um arquivo com todas as anotações.
df_prec['Token'] = df_prec['Token'].astype('str')

In [10]:
df_prec.head(10)

,Token,Tag
0,Súmula,B_Classe - Súmula
1,,I_Classe - Súmula
2,284,B_Número
3,/,I_Número
4,STF,B_Corte
5,Rcl,B_Classe - Writ
6,,I_Classe - Writ
7,11.022,B_Número
8,-,I_Número
9,ED,B_Tipo


In [11]:
# Tags de precedente
list(df_prec.Tag.unique())

['B_Classe - Súmula',
 'I_Classe - Súmula',
 'B_Número',
 'I_Número',
 'B_Corte',
 'B_Classe - Writ',
 'I_Classe - Writ',
 'B_Tipo',
 'I_Tipo',
 'O',
 'B_Relator',
 'I_Relator',
 'B_Data de Publicação',
 'B_Origem',
 'B_Classe - Recursal',
 'I_Classe - Recursal',
 'I_Origem',
 'B_Data de Julgamento',
 'I_Corte',
 'B_Classe - Penal originária',
 'I_Classe - Penal originária',
 'B_Classe - Outros',
 'I_Classe - Outros',
 'I_Data de Julgamento',
 'B_Classe - Constitucional',
 'I_Classe - Constitucional',
 'I_Data de Publicação',
 'B_Pessoa - Não Relator',
 'I_Pessoa - Não Relator']

In [14]:
df_prec.profile_report()

# Ideia para separação de sentenças:

-- Obter uma tag inicial que começa com B_, a sentença dura até que 'I_(fim da tag inicial)' se altere

In [17]:
combined_csv = pd.read_csv('preprocessados1.csv')
combined_csv.head(10)

,Token,Tag,Sentence #
0,EMENTA,INICIO_ARQ,Sentence 1
1,:,O,Sentence 1
2,,O,Sentence 1
3,CONSTITUCIONAL,O,Sentence 1
4,.,O,Sentence 1
5,,O,Sentence 1
6,TRIBUTÁRIO,O,Sentence 1
7,.,O,Sentence 1
8,,O,Sentence 1
9,IMPOSTO,O,Sentence 1


In [19]:
combined_csv[combined_csv.Tag.str.endswith('Precedente')].head(20)

,Token,Tag,Sentence #
64,ADI,B_Precedente,Sentence 2
65,,I_Precedente,Sentence 2
66,2.588,I_Precedente,Sentence 2
67,/,I_Precedente,Sentence 2
68,DF,I_Precedente,Sentence 2
815,RECURSO,B_Precedente,Sentence 9
816,,I_Precedente,Sentence 9
817,EXTRAORDINÁRIO,I_Precedente,Sentence 9
818,,I_Precedente,Sentence 9
819,541.090,I_Precedente,Sentence 9


In [25]:
# Conferindo se nao pode existir o caso onde uma Tag que começa com 'I' é seguida de outra Tag que começa
# com 'I'
lista = []
for i in range(len(df_prec)):
    if df_prec.Tag.iloc[i].startswith('I') == True and df_prec.Tag.iloc[i] != df_prec.Tag.iloc[i+1] == True:
        lista = df_prec.Tag.iloc[i+1]
#         print(df_prec.Tag.iloc[i+1])
lista

[]

In [26]:
df_prec.head(10)

,Token,Tag
0,Súmula,B_Classe - Súmula
1,,I_Classe - Súmula
2,284,B_Número
3,/,I_Número
4,STF,B_Corte
5,Rcl,B_Classe - Writ
6,,I_Classe - Writ
7,11.022,B_Número
8,-,I_Número
9,ED,B_Tipo


In [43]:
df_prec.Tag.iloc[0]#.startswith('B')

'B_Classe - Súmula'

In [38]:
df_prec.Tag.iloc[1][1:]

'_Classe - Súmula'

In [39]:
df_prec.Tag.iloc[0][1:] == df_prec.Tag.iloc[1][1:]

True

In [48]:
df_prec['Sentence'] = 'Sentence'

In [49]:
df_prec.head()

,Token,Tag,Sentence
0,Súmula,B_Classe - Súmula,Sentence
1,,I_Classe - Súmula,Sentence
2,284,B_Número,Sentence
3,/,I_Número,Sentence
4,STF,B_Corte,Sentence


In [50]:
for i in range(len(df_prec)):
    if df_prec.Tag.iloc[i].startswith('B'):
        if df_prec.Tag.iloc[i][1:] == df_prec.Tag.iloc[i+1][1:]:
            df_prec.Sentence.iloc[i] = 'Sentence %d'%i

KeyboardInterrupt: 

In [28]:
list(df_prec.Tag.unique())

['B_Classe - Súmula',
 'I_Classe - Súmula',
 'B_Número',
 'I_Número',
 'B_Corte',
 'B_Classe - Writ',
 'I_Classe - Writ',
 'B_Tipo',
 'I_Tipo',
 'O',
 'B_Relator',
 'I_Relator',
 'B_Data de Publicação',
 'B_Origem',
 'B_Classe - Recursal',
 'I_Classe - Recursal',
 'I_Origem',
 'B_Data de Julgamento',
 'I_Corte',
 'B_Classe - Penal originária',
 'I_Classe - Penal originária',
 'B_Classe - Outros',
 'I_Classe - Outros',
 'I_Data de Julgamento',
 'B_Classe - Constitucional',
 'I_Classe - Constitucional',
 'I_Data de Publicação',
 'B_Pessoa - Não Relator',
 'I_Pessoa - Não Relator']